# Introduction of Deep Learning

## Some math knowledge can help you understand the machine leanring algorithms better:
 * Calculus
 * Linear Algebra
 * Statistics

## We utilize [Python](https://www.python.org/) packags to build the deep learning architectures.
 * [NumPy](https://numpy.org/)
 * [PyTorch](https://pytorch.org/)
 
## You can program and execute via [Google Colab](https://colab.research.google.com). Google Colab provids free GPU and TPU! 

## Intro To Convolutional Neural Networks (CNNs)
### Handwritten Digit Recognition Using PyTorch

We will use the most popular task in computational vision, [MNIST database](http://yann.lecun.com/exdb/mnist/). It is a collection of 70,000 handwritten digits split into TRAIN and TEST set of 60,000 and 10,000 images respectively.

![](https://camo.githubusercontent.com/d440ac2eee1cb3ea33340a2c5f6f15a0878e9275/687474703a2f2f692e7974696d672e636f6d2f76692f3051493378675875422d512f687164656661756c742e6a7067)

Picture Courtesy: https://camo.githubusercontent.com/d440ac2eee1cb3ea33340a2c5f6f15a0878e9275/687474703a2f2f692e7974696d672e636f6d2f76692f3051493378675875422d512f687164656661756c742e6a7067

## Import required Python libraries

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from tqdm import tqdm, trange

We will define a constant to decide whether to use the GPU (with CUDA specifically) or the CPU. 

If you don't have a GPU, set this is CPU. Later when we create tensors, this variable will be used to decide whether we keep them on CPU or move them to GPU.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


`PyTorch` provides an easy implementation to download the cleaned and already prepared `MNIST data`.

Before downloading the data, we should define what are the transformations we want to perform on the image data.

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((1,), (1,)),
                              ])

1. **`transforms.Compose`** - Composes several transforms together.
2. **`transforms.ToTensor( )`** — Convert a `PIL Image` or `numpy.ndarray` to `tensor`, that are understandable for the computational method.
2. **`transforms.Normalize( )`** — Normalize a tensor image with mean and standard deviation. Our MNIST dataser is black-white pictures that only have one channel.

### Now, we download the data sets, shuffle them and transform images to tensor. 

We load datasets to **`DataLoader`** to generate batches. Then, we save the transformed dataset in directory `./image_data/`.

In [ ]:
batch_size = 64
trainset = datasets.MNIST('./image_data/', download=True, train=True, transform=transform)
valset = datasets.MNIST('./image_data/', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=True)

Extracting ./image_data/MNIST/raw/train-images-idx3-ubyte.gz to ./image_data/MNIST/raw


Extracting ./image_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./image_data/MNIST/raw



Extracting ./image_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./image_data/MNIST/raw


Extracting ./image_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./image_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Convolutional Neural Network (CNN)

![](https://www.frontiersin.org/files/Articles/273835/fpsyg-08-01745-HTML/image_m/fpsyg-08-01745-g001.jpg)

Picture Courtesy: https://www.frontiersin.org/articles/10.3389/fpsyg.2017.01745/full

1:- **Convolution:**
The first thing of CNN is the actual convolution part. Convolution is a **moving kernel (or filter)** across the image being studied.

2:- **ReLU:** The next step in the CNN structure is to pass the output of the convolution operation through **a non-linear activation function**. 

3:- **Pooling:** The third element in CNN is **Max Pooling**.

4:- **Fully Connected Layers:** The **Sigmoid fully connected layer** will give the prediction probilities across all labels.

5:- **Dropout:** To regularize the model, we can use dropout in convolution layers, pooling layers, or fully connected layers.

## Build CNN with PyTorch

Load Python packages

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Creating CNN class

Let's put everything to a `class` object. 

1. We create conv_layer1 (`self.conv_layer1`) by creating a `nn.Sequential` object. 

   This method is a handy way of creating a computational sequence: **convolution -> ReLU -> pooling**. We put these module to the `nn.Sequential` object, sequentially.

2. Using the same logic, we create conv_layer2 (`self.conv_layer2`) where also includes a same computational sequence.

3. Then, we add a dropout layer and two fully connected layers. 

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.conv_layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)
    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = F.sigmoid(self.fc1(out))
        out = self.fc2(out)
        return out

### Training the model

We create an instance of our ConvNet class, and define our loss function and optimizer:

In [ ]:
model = ConvNet().to(device)
learning_rate = 0.001
# Loss and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

**Calculate the number of parameters.**

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,199,106 trainable parameters


`train()` and `evaluate()` functions

**`train()`** function takes data iterator, model, model optomizer, and loss function as inputs. The input image is passed through the CNN model. CNN model outputs the predictions of input samples. Loss function computes the training loss between predication and true label. We then use the loss to back-propagate through CNN model and update the model weights.



In [ ]:
def train(model, iterator, optimizer, criterion):
    
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        images = batch[0].to(device)
        labels = batch[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)

        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.cpu().item()

    return epoch_loss / len(iterator)

**`evaluate()`** function takes data iterator, model, and loss function as inputs. The input image is passed through the CNN model. Loss function computes the evaluation loss between predication and true label. We also compute the the predition accuracy and $F_1$ score.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    all_pred=[]
    all_label = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            images = batch[0].to(device)
            labels = batch[1].to(device)

            optimizer.zero_grad()

            outputs = model(images)

            loss = criterion(outputs, labels)

            epoch_loss += loss.cpu().item()

            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(labels.cpu())
    
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    return epoch_loss / len(iterator), accuracy, f1score

### Training the model

In [ ]:
MAX_EPOCH = 5  # the number of passes of the entire training dataset
total_step = len(trainloader)
loss_list = []
acc_list = []

for epoch in trange(MAX_EPOCH, desc="Epoch"):
    train_loss = train(model, trainloader, optimizer, loss_func)  
    val_loss, val_acc, val_f1 = evaluate(model, valloader, loss_func)

    print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, MAX_EPOCH, train_loss, val_loss, val_acc, val_f1))
    


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")

Epoch:  20%|██        | 1/5 [00:16<01:05, 16.47s/it]


 Epoch [1/5], Train Loss: 0.0936, Validation Loss: 0.0437, Validation Accuracy: 0.9866, Validation F1: 0.9866



Epoch:  40%|████      | 2/5 [00:32<00:49, 16.48s/it]


 Epoch [2/5], Train Loss: 0.0724, Validation Loss: 0.0366, Validation Accuracy: 0.9879, Validation F1: 0.9878



Epoch:  60%|██████    | 3/5 [00:49<00:32, 16.38s/it]


 Epoch [3/5], Train Loss: 0.0593, Validation Loss: 0.0297, Validation Accuracy: 0.9898, Validation F1: 0.9897



Epoch:  80%|████████  | 4/5 [01:05<00:16, 16.30s/it]


 Epoch [4/5], Train Loss: 0.0516, Validation Loss: 0.0318, Validation Accuracy: 0.9886, Validation F1: 0.9885



Epoch: 100%|██████████| 5/5 [01:21<00:00, 16.32s/it]


 Epoch [5/5], Train Loss: 0.0451, Validation Loss: 0.0249, Validation Accuracy: 0.9919, Validation F1: 0.9918


### Visualization 
We also introduce a online tool to visualize the CNN model. 

* Visualizing the “activations” of the layers: https://cs.ryerson.ca/~aharley/vis/conv/

![](http://www.programmersought.com/images/544/46bde7423a265a6ba287620bc5c47f68.png)
Picture Courtesy: http://www.programmersought.com/article/23607351/

**Reference:**
* https://pytorch.org/docs/stable/index.html
* https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/
* https://nextjournal.com/gkoehler/pytorch-mnist
* https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627
* https://algorithmia.com/blog/convolutional-neural-nets-in-pytorch
* https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
* http://cs231n.stanford.edu/